In [1]:
# import necessary functions/pacakges/libraries 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dash
import altair as alt
from dash import dcc, html
import plotly.express as px
import folium as fm
from folium.plugins import HeatMapWithTime
import plotly.graph_objects as go
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import warnings
import dash_vega_components as dvc
import plotly.tools as tls
warnings.filterwarnings("ignore")

In [17]:
# read in data and create a dataframe
df = pd.read_csv('Traffic_Records.csv')

In [3]:
df

,Published_Date,Traffic_Report_ID,Issue_Reported,Latitude,Longitude,Address,AM_PM
0,2021-01-01 00:01:11+00:00,899A19B6BEC5CDDC50179F183BA138B628CF94B3_16094...,CRASH URGENT,30.222103,-97.676940,1500-1578 E Sh 71 Eb,AM
1,2021-01-01 00:15:48+00:00,45CE120FEAA2C1A2BB53DB6C8FB833E58D6BB661_16094...,CRASH URGENT,30.357592,-97.686939,801-829 E Rundberg Ln,AM
2,2021-01-01 00:29:25+00:00,F673929B7AE05128DD473C556386A7B585483947_16094...,STALLED VEHICLE,30.218082,-97.690595,E Ben White Blvd Svrd Eb & E Riverside Dr,AM
3,2021-01-01 00:49:45+00:00,DF1FA50C9F190117BEC623D72ED64F655EA0360F_16094...,COLLISION,30.164672,-97.724830,7201-7241 MC KINNEY FALLS PKWY,AM
4,2021-01-01 00:55:29+00:00,7C9FE6831F52E30E0EDE4F8C54FD9BBA673E8D8B_16094...,CRASH SERVICE,30.373617,-97.677882,11105 N Ih 35 Svrd Nb,AM
...,...,...,...,...,...,...,...
184608,2024-08-16 03:48:34+00:00,DB8DC2B7A3BB754178D96FE9B4E434D0C017825B_17237...,TRAFFIC HAZARD,30.263114,-97.705584,3017 Lyons Rd,AM
184609,2024-08-16 04:33:43+00:00,8639C2E8C44D79A5AFAF095AA46F9C9231829B72_17237...,TRAFFIC HAZARD,30.367928,-97.718562,8817-9009 Research Blvd Nb,AM
184610,2024-08-16 04:36:35+00:00,23D202FB561C67AC5D22AE22F0E595ED35106B02_17237...,CRASH URGENT,30.205539,-97.716533,Montopolis Dr / Burleson Rd,AM
184611,2024-08-16 05:01:51+00:00,A031EB6D2F6330F89B937098D4439578421617DF_17237...,CRASH URGENT,30.338738,-97.682241,Cameron Rd / Park Center Dr,AM


In [19]:
# set index as datetime
df.set_index('Published_Date', inplace=True)

In [21]:
# create column datetime
df['datetime'] = df.index
df['datetime'] = pd.to_datetime(df['datetime'])

# create column hour
df['hour'] = df['datetime'].dt.hour

## Develop the Dashboard with Dash

In [35]:
# create dash app 
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUMEN])

# token to access Mapbox maps
mapbox_access_token = 'pk.eyJ1Ijoidm4xODg5ODQiLCJhIjoiY20zNjM2OGlrMDE1ODJqb2NjdzIxeDkwNyJ9.V5zQ5VY87nm5V1t-5qSdXQ'

# static graphs ####################################################################################################################################
def incidents_by_hour(df):
    # Assuming df1 is your dataframe
    # Count the number of accidents per hour
    accidents_by_hour = df['hour'].value_counts().sort_index()

    # Create the Plotly figure
    fig = go.Figure()
    
    # Add line plot
    fig.add_trace(go.Scatter(
        x=accidents_by_hour.index,  # x-values (hours)
        y=accidents_by_hour.values,  # y-values (number of accidents)
        mode='lines+markers',  # Display both line and markers
        line=dict(color='skyblue', width=2),  # Line style
        marker=dict(color='skyblue', size=8),  # Marker style
    ))
    
    # Add title and labels
    fig.update_layout(
        title='Traffic Incidents in Austin by Hour of the Day',
        title_font=dict(size=20, color='black'),
        font=dict(size=12, color='black'),
        xaxis_title='Hour of the Day (CST)',
        yaxis_title='Number of Incidents',
        yaxis=dict(
            dtick=2000,  # Set the interval between ticks to 5
            showgrid=True,
            gridcolor='lightgrey',  # Set gridline color (you can change this)
            zeroline=True,  # Show the line at x=0 (if applicable)
            zerolinecolor='black',
        ), 
        plot_bgcolor='white',
        xaxis=dict(
        showgrid=False,  # Enable gridlines for the x-axis
        gridcolor='lightgrey',  # Set gridline color (you can change this)
        zeroline=True,  # Show the line at x=0 (if applicable)
        zerolinecolor='black',
        tickmode='array', tickvals=list(range(0, 24))# Ensure x-axis has all hours (0-23)
        )
    )
    return fig

def incident_type_frequency(df):
    issue_counts = df['Issue_Reported'].value_counts().reset_index()
    issue_counts.columns = ['Issue_Reported', 'Frequency']
    
    # Create a selection for hover
    highlight = alt.selection_point(on='mouseover', empty= False, fields=['Issue_Reported'])
    
    # Create the bar chart with hover effect
    chart = alt.Chart(issue_counts).mark_bar(color='steelblue').encode(
        x=alt.X('Issue_Reported', sort='-y', title='Type of Accident'),
        y=alt.Y('Frequency', title='Frequency'),
        tooltip=['Issue_Reported', 'Frequency'],
        color=alt.condition(highlight, alt.value('red'), alt.value('Green'))
    ).interactive(
    ).add_params(
        highlight
    ).properties(
        title='Frequency of Each Type of Incident',
        width=600,
        height=220
    ).configure_axis(
        labelAngle=45
    )
    
    return chart

def incidents_by_dayofweek(df):
    # Group by day and issue type and count the incidents
    df = pd.read_csv('Traffic_Records.csv')
    df['Published_Date'] = pd.to_datetime(df['Published_Date'])
    df['Day'] = df['Published_Date'].dt.weekday
    day_trends = df.groupby(['Day', 'Issue_Reported']).size().reset_index(name='Count')
    
    # Mapping of day numbers (0-6) to weekday names
    day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    day_trends['Day'] = day_trends['Day'].apply(lambda x: day_names[x])
    
    # Create the stacked bar chart using Plotly Express
    fig = px.bar(
        day_trends,
        x='Day',
        y='Count',
        color='Issue_Reported',
        title='Traffic Incidents: Day of the Week & Accident Type',
        labels={'Count': 'Number of Incidents', 'Day': 'Day of the Week', 'Issue_Reported': 'Type of Incident'},
        color_discrete_sequence=px.colors.qualitative.Set2,  # Optional: change color scheme
        barmode='stack',  # Stack the bars
    )

    fig.update_layout(
        title_font=dict(size=20, color='black'),
        font=dict(size=12, color='black'),
        yaxis=dict(
            dtick=2000,  # Set the interval between ticks to 5
            showgrid=True,
            gridcolor='lightgrey',  # Set gridline color (you can change this)
            zeroline=True,  # Show the line at x=0 (if applicable)
            zerolinecolor='black',
        ), 
        plot_bgcolor='white',
        xaxis=dict(
        showgrid=False,  # Enable gridlines for the x-axis
        gridcolor='lightgrey',  # Set gridline color (you can change this)
        zeroline=True,  # Show the line at x=0 (if applicable)
        zerolinecolor='black',
        tickmode='array', tickvals=list(range(0, 24))# Ensure x-axis has all hours (0-23)
        )
    )
    
    # Show the plot
    return fig


def incident_piechart(df):
    # Load the dataset
    traffic_data = pd.read_csv('Traffic_Records.csv')
    traffic_data['Published_Date'] = pd.to_datetime(traffic_data['Published_Date'])

    # Extract month from the Published_Date
    traffic_data['Month'] = traffic_data['Published_Date'].dt.month

    # Define the season mapping
    def get_season(month):
        if month in [12, 1, 2]:
            return 'Winter'
        elif month in [3, 4, 5]:
            return 'Spring'
        elif month in [6, 7, 8]:
            return 'Summer'
        elif month in [9, 10, 11]:
            return 'Fall'

    # Apply the season mapping to the 'Month' column
    traffic_data['Season'] = traffic_data['Month'].apply(get_season)

    # Count incidents by season
    season_incidents = traffic_data['Season'].value_counts()

    # Define color scale: a soft green-yellow transition (as in your original code)
    colors = ['#d9f0a3', '#8cce3e', '#468e23', '#1d661b'][::-1]  # Custom green shades for soft transitions

    # Plot the pie chart using Plotly
    fig = go.Figure(data=[go.Pie(
        labels=season_incidents.index,
        values=season_incidents.values,
        hoverinfo='label+value',  # Show label and percentage
        textinfo='label+percent',  # Show label and percentage on the chart
        marker=dict(colors=colors),  # Custom colors
        pull=[0.1, 0, 0, 0],  # Optional: offset the first slice for a 3D effect
    )])

    fig.update_layout(
        title='Total Traffic Incidents by Season',
        showlegend=True,
        title_font=dict(size=20, color='black'),
        font=dict(size=12, color='black')
    )
    return fig
    



# title ############################################################################################################################################
app.layout = dbc.Container([
    # header
    dbc.Row(
        dbc.Col(html.H1("Traffic Incidents in Austin (2022-2024)", className="display-10"), width=12),
    className="mb-4"),
    
### visual box 1 ###################################################################################################################################
    dbc.Row([
        dbc.Col(html.Label('Select an Hour:', style={'height': '1px', 'fontWeight': 'bold', 'color': 'black'}), width=1, className="mb-2"),
        dbc.Col(dcc.Slider(
            0, 23, 1, id='hour-slider', value=0,
            marks={i: str(i) for i in range(24)}, included=False,
            tooltip={'always_visible': False, 'placement': 'top'},
        ), style={
                'paddingLeft': '0',  
                'textAlign': 'left',
                'boxShadow': '0px 4px 10px rgba(0,0,0,0.3)',
                'border': '5px solid white',
                'borderRadius': '5px'
            }, width=7, className="mb-2"),
    ]),
    
    dbc.Row([
        # graph
        dbc.Col(dcc.Graph(id='map', style={
            'height': '670px', 
            'boxShadow': '0px 4px 10px rgba(0,0,0,0.3)',
            'border': '5px solid white',
            'borderRadius': '5px'
        }), width=8, style={'paddingRight': '1px'}, className="mb-2"),

        # incidents barchart by hour selected
        dbc.Col(
            dbc.Stack([
                dcc.Graph(id='incident-bar-chart', style={
                'height': '570px', 
                'boxShadow': '0px 4px 10px rgba(0,0,0,0.3)',
                'border': '5px solid white',
                'borderRadius': '5px' 
                }, className="mb-2"),
                
                dbc.Col(html.H1('Total Incidents:', id='total-incidents', 
                                style={'fontWeight': 'bold', 
                                       'color': 'black',
                                       'textAlign': 'center',
                                       'boxShadow': '0px 4px 10px rgba(0,0,0,0.3)',
                                       'border': '5px solid white',
                                       'borderRadius': '5px',
                                       'height': '91px',
                                       'paddingTop': '15px'
                                       
                                        }))
            ])
        )
    ], justify="start"),

    dbc.Row([
        dbc.Col(html.Label('Select Incident Type(s):', style={'height': '1px', 'fontWeight': 'bold', 'color': 'black'}), width=1),
        dbc.Col(dcc.Checklist(
            id='incident-type-checklist',
            options=[{'label': incident, 'value': incident} for incident in df['Issue_Reported'].unique()],
            value=[], inline=True,
            labelStyle={'marginRight': '10px', 'fontSize': '12px', 'color': 'black'}
        ),
        width=7, 
        style={
            'boxShadow': '0px 4px 10px rgba(0,0,0,0.3)',
            'border': '5px solid white',
            'borderRadius': '5px' 
        }),
    ],className="mb-2"),

    dbc.Row([
        dbc.Col(width=1),
        dbc.Col(html.Button('Select All', id='select-all', n_clicks_timestamp=-1), width='auto'),
        dbc.Col(html.Button('Deselect All', id='deselect-all', n_clicks_timestamp=-1), width='auto'),
    ], className="mb-4"),
####################################################################################################################################################


## visual box 2 ####################################################################################################################################
    dbc.Row([
        dbc.Col(
            dcc.Graph(
                id='incidents-by-hour',
                figure = incidents_by_hour(df), 
                style={ 
                'height': '450px', 
                'boxShadow': '0px 4px 10px rgba(0,0,0,0.3)',
                'border': '5px solid white',
                'borderRadius': '5px',
                }
            ), 
            width=5, style={'paddingRight': '1px'}
        ),
        
        dbc.Col(
            dcc.Graph(
                id='incidents-piechart',
                figure = incident_piechart(df), 
                style={ 
                'height': '450px', 
                'boxShadow': '0px 4px 10px rgba(0,0,0,0.3)',
                'border': '5px solid white',
                'borderRadius': '5px',
                }
            ), 
            width=5
        )
    ], className="mb-4", style={'paddingRight': '1px'}),

####################################################################################################################################################
    # side by side visual (small graphs)
    dbc.Row([
        dbc.Col(
            dcc.Graph(
                id='incidents-by-weekday', 
                figure = incidents_by_dayofweek(df),
                style={'height': '450px', 
                       'boxShadow': '0px 4px 10px rgba(0,0,0,0.3)',
                       'border': '5px solid white',
                       'borderRadius': '5px',
                      }
            ), 
            width=6, style={'paddingRight': '1px'}),
        
        dbc.Col(
            dbc.Stack([
                    dcc.Dropdown(
                        id='incident-type-dropdown',
                        options=[{'label': incident, 'value': incident} for incident in df['Issue_Reported'].unique()],
                        value='CRASH URGENT',
                        style={'width':'300px'}
                    ),

                    dcc.Graph(
                        id='incident-line-chart',
                        style={'height': '414px', 
                               'boxShadow': '0px 4px 10px rgba(0,0,0,0.3)',
                               'border': '5px solid white',
                               'borderRadius': '5px'}
                    ), 
            ]),
        )
       
    ], className="mb-4"),

    # last full width visual
    
    dbc.Row([
        dbc.Col(html.Label('Incident Heatmap for Austin', 
                       style={'fontSize': '25px', 
                              'color': 'black',
                              'textAlign': 'center', 
                              'marginBottom': '10px'}), 
            width=12),  # Adjust width if necessary
        
        # graph
        dbc.Col(html.Iframe(id='heatmap', srcDoc= open('IncidentHeatMap.html', 'r').read(),  
            style={
                'width': '900px', 
                'height': '600px',
                'boxShadow': '0px 4px 10px rgba(0,0,0,0.3)',
                'border': '20px solid white',
                'borderRadius': '5px',
                'marginRight': '10px'
            },
        ), 
        ),

    ]),

# whole page styling
], style={
        'backgroundColor': 'white',  
        'border': '5px solid white',  # White border inside
        'borderRadius': '20px',  # Smooth corners
        'boxShadow': '0px 0px 30px 5px rgba(0,0,0,0.5)',  # Outer green border effect (shadow)
        'margin': 'auto'
    }, fluid=True)

#### callbacks and updates #########################################################################################################################

# convert 24 hour time to 12 hour time
def hour24_to_hour12(hour_value):
    return hour_value if hour_value != 0 and hour_value <= 12 else abs(hour_value - 12) if hour_value != 12 else hour_value

# callbacks for select/deselect all
@app.callback(
    Output('incident-type-checklist', 'value'),
    [Input('select-all', 'n_clicks_timestamp'), 
     Input('deselect-all', 'n_clicks_timestamp')]
)

def update_checklist(select_time, deselect_time):
    all_incidents = df['Issue_Reported'].unique()
    return all_incidents.tolist() if select_time > deselect_time else []

# callback to update map based on hour and selected incident types
@app.callback(
    Output('map', 'figure'),
    [Input('hour-slider', 'value'), Input('incident-type-checklist', 'value')]
)

def update_map(hour_value, selected_incidents):
    filtered_df = df[(df['hour'] == hour_value) & (df['Issue_Reported'].isin(selected_incidents))]
    suffix = 'AM' if hour_value < 12 else 'PM'
    hour_display = hour24_to_hour12(hour_value)
    
    fig = go.Figure(go.Scattermapbox(
        lat=filtered_df['Latitude'], lon=filtered_df['Longitude'],
        mode='markers', marker=go.scattermapbox.Marker(size=5, color='red'),
        text=filtered_df['Issue_Reported'], hoverinfo='text'
    ))

    fig.update_layout(
        mapbox_style='mapbox://styles/mapbox/streets-v11', mapbox_zoom=9,
        mapbox_center={"lat": 30.266666, "lon": -97.733330},
        mapbox_accesstoken=mapbox_access_token,
        title=dict(text=f"Traffic Incidents for {hour_display}:00 {suffix} (CST)", xref='paper', x=0, y=0.99),
        title_font=dict(size=20, color='black'),
        margin={'r': 20, 't': 30, 'l': 20, 'b': 20}
    )
    return fig

# callback to update bar chart
@app.callback(
    Output('incident-bar-chart', 'figure'),
    [Input('hour-slider', 'value')]
)

def update_bar_chart(hour_value):
    # filter and count incidents
    filtered_df = df[(df['hour'] == hour_value)]
    incident_counts = filtered_df['Issue_Reported'].value_counts().reset_index()
    incident_counts.columns = ['Incident Type', 'Count']

    # Create a horizontal bar chart
    fig = go.Figure(go.Bar(
        y=incident_counts['Incident Type'],
        x=incident_counts['Count'],
        orientation='h',
        marker=dict(color='Maroon')
    ))

    
    fig.update_layout(
        title=dict(text=f'Incidents by Type for Hour {hour_value}:00'),
        title_font=dict(size=20, color='black'),
        xaxis_title='Number of Incidents',
        yaxis_title='Incident Type',
        font=dict(size=12, color='black'),
        margin_r = 20,
        margin_l = 20,
        plot_bgcolor='white',
        xaxis=dict(
        showgrid=True,  # Enable gridlines for the x-axis
        gridcolor='lightgrey',  # Set gridline color (you can change this)
        zeroline=True,  # Show the line at x=0 (if applicable)
        zerolinecolor='black',  # Set color of the zero line (optional)
        ),
    )

    return fig

@app.callback(
    Output('total-incidents', 'children'),
    [Input('hour-slider', 'value')]
)

def update_total_incidents(hour_value):
    # filter incidents
    filtered_df = df[(df['hour'] == hour_value)]
    total_incidents = filtered_df['Issue_Reported'].count()
    return f'Total Incidents: {total_incidents}'

@app.callback(
    Output('incident-line-chart', 'figure'),
    [Input('incident-type-dropdown', 'value')]
)

def update_incident_line_chart(incident_type):
    df = pd.read_csv('Traffic_Records.csv')
    # Convert the Published_Date to datetime format and extract the date
    df['Published_Date'] = pd.to_datetime(df['Published_Date'])
    df['Date'] = df['Published_Date'].dt.date  
    
    # Group by date and type of accident, counting the number of incidents
    daily_trends = df.groupby(['Date', 'Issue_Reported']).size().reset_index(name='Count')
    
    # Filter out rows where Count is 0 or Issue_Reported is missing
    daily_trends = daily_trends[daily_trends['Count'] > 0].dropna(subset=['Issue_Reported'])
    daily_trends = daily_trends[daily_trends['Issue_Reported'] == incident_type]
    
    # Create the line chart
    fig = px.line(
        daily_trends,
        x='Date',
        y='Count',
        color='Issue_Reported',
        title='Daily Trend of Traffic Incidents by Selected Type',
        labels={'Date': 'Date', 'Count': 'Number of Incidents', 'Issue_Reported': 'Type of Incident'},
        hover_data={'Count': True, 'Date': '|%B %d, %Y'},
        color_discrete_sequence= px.colors.qualitative.Set2  # Apply warm colors to the lines
    )

    fig.update_layout(
        showlegend=False,
        title_font=dict(size=20, color='black'),
        font=dict(size=12, color='black')
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=False, use_reloader=False)